# Deep Learning - Neural Network From Scratch

To recognize individual digits we will use a three-layer neural network:

![neural-network-for-digits](../images/neural-network-for-digits.PNG)

The input layer of the network contains neurons encoding the values of the input pixels. Our training data for the network will consist of many 28 by 28 pixel images of scanned handwritten digits, and so the input layer contains 784=28×28 neurons. The input pixels are greyscale, with a value of 0.0 representing white, a value of 1.0 representing black, and in between values representing gradually darkening shades of grey.

The second layer of the network is a hidden layer. We denote the number of neurons in this hidden layer by n, and we'll experiment with different values for n. The example shown illustrates a small hidden layer, containing just n=15 neurons.

The output layer of the network contains 10 neurons. If the first neuron fires, i.e., has an output ≈ 1, then that will indicate that the network thinks the digit is a 0. If the second neuron fires then that will indicate that the network thinks the digit is a 1. And so on. A little more precisely, we number the output neurons from 0 through 9, and figure out which neuron has the highest activation value. If that neuron is, say, neuron number 6, then our network will guess that the input digit was a 6. And so on for the other output neurons.

-----------------------------------------------

## Learning with Gradient Descent

We'll use the notation $\vec{x}^{\,}$ to denote a training input. It'll be convenient to regard each training input $\vec{x}^{\,}$ as a 28×28=784 - dimensional vector. Each entry in the vector represents the grey value for a single pixel in the image. The desired output will be $\vec{y}^{\,}(\vec{x}^{\,})$ where $\vec{y}^{\,}$ is a 10 - dimensional vector. For instance, if $\vec{x}^{\,}$ is a 28x28=784 picture representing 7, then the ideal output of the network should be  $\vec{y}^{\,}(\vec{x}^{\,}) = [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]^T$

What we'd like is an algorithm that lets us find weights and biases so that the output from the network is as close as possible to the ideal output $\vec{y}^{\,}(\vec{x}^{\,})$ for all training inputs $\vec{x}^{\,}$. To quantify how well we're achieving this goal we define a **cost function**. The cost function should output a high value (scalar) if our network is not sure how to recognize digits and low value when the network is confident that it can recognize digits correctly.

$$C(\vec{w}^{\,},\vec{b}^{\,}) = \frac{1}{2n}\sum_x{||\vec{y}^{\,}(\vec{x}^{\,}) - \vec{a}^{\,}(\vec{x}^{\,},\vec{w}^{\,},\vec{b}^{\,})||^2}$$

The function above can be our cost function for this example. In this function, $\vec{w}^{\,}$ represents all weights in the network, $\vec{b}^{\,}$ represents all biases in the network, $\vec{a}^{\,}$ is the vector of outputs from the network when $\vec{x}^{\,}$ is input, and the sum is over all training inputs, $\vec{x}^{\,}$. We'll call $C$ _the quadratic cost function_; it's also sometimes known as the _mean squared error_ or just **MSE**.

The cost $C(\vec{w}^{\,},\vec{b}^{\,})$ becomes small, i.e., $C(\vec{w}^{\,},\vec{b}^{\,})≈0$, precisely when $\vec{y}^{\,}(\vec{x}^{\,})$ is approximately equal to the output, $\vec{a}^{\,}$, for all training inputs, $\vec{x}^{\,}$. So our training algorithm has done a good job if it can find weights and biases so that $C(\vec{w}^{\,},\vec{b}^{\,})≈0$. By contrast, it's not doing so well when $C(\vec{w}^{\,},\vec{b}^{\,})$ is large - that would mean that $\vec{y}^{\,}(\vec{x}^{\,})$ is not close to the output $\vec{a}^{\,}$ for a large number of inputs. So the aim of our training algorithm will be to minimize the cost $C(\vec{w}^{\,},\vec{b}^{\,})$ as a function of the weights and biases. In other words, we want to find a set of weights and biases which make the cost as small as possible. We'll do that using an algorithm known as **gradient descent**.

To explain the intuition behind gradient descent we will start with the function $C(x,y) = x^2 + y^2$. The gradient is a vector of all partial derivatives of a function:

$$\nabla C(x,y) = [\frac{\delta C(x,y)}{\delta x}, \frac{\delta C(x,y)}{\delta y}]$$

When we calculate the gradient we get:

$$\nabla C(x,y) = [2x, 2y]$$

What does the gradient tell us? When we are at the point $(x,y)$ it explains in which **DIRECTION (not value)** $(2x, 2y)$ we need to go in order to maximize the impact on the function $C(x, y)$. Hence, when we take the negative gradient we are going in a direction to minimize the loss/cost function.

How does this translate to our neural network? Well, our cost function $C(\vec{w}^{\,},\vec{b}^{\,})$ has a lot of parameters but in the end, it's just a multivariable function. We can randomly choose $\vec{w}^{\,}$ and $\vec{b}^{\,}$ and calculate the gradient at that point of a cost function. The gradient will tell us in which direction we should go in order to reduce the value of a cost function. In order to go downhill, we will need some learning step that will represent how fast we want to go downhill. If steps are small, then learning process will be slow. If steps are high, then we might never find a minimum.

The gradient is actually averaged over all training examples. This means that we calculate gradient for one training example $\vec{x_1}^{\,}$ and note what changes of $\vec{w}^{\,}$ and $\vec{b}^{\,}$ should be in order to improve the cost function for this example. We need to do this for every training example and find the average value of individual changes in order to make a new step in the cost function. This process is costly. So, we are going to use **stochastic gradient descent** that will take batches of input training data and make gradient descent based on those batches (we are essentially making an approximation of a true gradient using this batch approach). Of course, the estimate won't be perfect - there will be statistical fluctuations - but it doesn't need to be perfect: all we really care about is moving in a general direction that will help decrease C, and that means we don't need an exact computation of the gradient. In practice, stochastic gradient descent is a commonly used and powerful technique for learning in neural networks.

In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt

### Implementing Neural Network From Scratch

We are going to use sigmoid neurons in this neural network. To calculate activations in the second layer of the network we use:

$$a_2 = \sigma(w*a_1 + b)$$

where:

- $w$ is a matrix of weights between first and the second layer of a network
- $a_1$ is a vector of activations in the input layer
- $b$ is a vector of biases for the sigmoid neurons in the second layer

For instance, if our network has 10 neurons in the input layer 7 neurons in the second layer, and 5 neurons in the output layer (a 3 layered network) then to calculate activations in the second layer we use:

$$a'_{7x1} = \sigma(w_{7x10}a_{10x1} + bias_{7x1})$$

Finally, for the output layer we have:

$$a''_{5x1} = \sigma(w_{5x7}a'_{7x1} + bias_{5x1})$$

In [2]:
class NeuralNetwork(object):
    """
    This class encapsulates neural network mechanics.
    """
    
    def __init__(self, sizes):
        """
        NeuralNetwork constructor.
        
        Attributes:
        -----------
        sizes: array that represents number of neurons in different layers of the network
        """
        self.number_of_layers = len(sizes)
        self.sizes = sizes
        self.weights = [np.random.randn(y, x) for x,y in zip(sizes[:-1], sizes[1:])]
        self.biases = [np.random.randn(x, 1) for x in sizes[1:]]
        
        
    def learn(self, training_data, epochs, batch_size, learning_rate, test_data=None):
        """
        Performs a neural network learning process.
        
        Attributes:
        -----------
        training_data: training data for the neural network in the form (x, y), 
        representing the training inputs and the desired outputs
        
        epochs: number of epochs to train the neural network
        batch_size: size of the mini batch
        learning_rate: learning rate for the steepest descent
        test_data: if available it is used to see how well the network is learning at each epoch
        """
        if test_data: size_test = len(test_data)
        size_training = len(training_data)
        
        # train a net for a specified number of epochs
        for i in range(epochs):
            random.shuffle(training_data)
            batches = [training_data[k:k+batch_size] for k in range(0, size_training, batch_size)]
            
            # do SGD per batch
            for batch in batches:
                self._sgd_step(batch, learning_rate)
            
            # show the results of a step if test data is available
            if test_data:
                print(f"Epoch {i}: {self.evaluate(test_data)} / {size_test}")
            # otherwise, just notify that epoch is completed
            else:
                print(f"Epoch {i} complete")
                
                
    def _sgd_step(self, batch, learning_rate):
        """
        Performs one gradient descent step for the batch.
        
        Attributes:
        -----------
        batch: subset of the training data in the form (x,y)
        learning_rate: learning rate for the steepest descent
        """
        # create empty gradients
        nabla_w = [np.zeros(w_matrix.shape) for w_matrix in self.weights]
        nabla_b = [np.zeros(b_vector.shape) for b_vector in self.biases]
        
        # for each data point in the batch 
        # update the gradient according to the backpropagation algorithm
        for x,y in batch:
            delta_nabla_b, delta_nabla_w = self._backpropagation(x, y)
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            
        # finally update the parameters of the network (weights and biases)
        self.weights = [w - (learning_rate/len(batch))*nw for w,nw in zip(self.weights, nabla_w)]
        self.biases = [b - (learning_rate/len(batch))*nb for b,nb in zip(self.biases, nabla_b)]
            
    def _backpropagation(self, x, y):
        """
        This function executes a backpropagation algorithm for one training example.
        
        Attributes:
        -----------
        x: activations in the input layer of a neural network
        y: desired output of a neural network
        """
        # gradient of weights and biases
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        
        # feedforward pass
        activations = [x]
        zs = []
        
        for w,b in zip(self.weights, self.biases):
            z = np.dot(w, activations[-1]) + b
            activations.append(self._sigmoid(z))
            zs.append(z)
        
        # backpropagation pass
        print(zs[-1], self._cost_derivative(activations[-1], y))
        
        delta = self._cost_derivative(activations[-1], y)*self._sigmoid_derivative(zs[-1])
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        nabla_b[-1] = delta
        
        for l in range(2, self.number_of_layers):
            z = zs[-l]
            sp = self._sigmoid_derivative(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            print(delta.shape, activations[-l-1].shape)
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
            
        return (nabla_w, nabla_b)
        
        
    
    def _feedforward(self, x):
        """
        Calculates the output of a neural network by feeding forward information 
        from the input layer to the output layer through sigmoid neurons.
        
        Attributes:
        -----------
        x: activations in the input layer of a neural network. They should be represented in the form of a vector (n, 1)
        """
        for w,b in zip(self.weights, self.biases):
            x = self._sigmoid(np.dot(w, x) + b) 
            
        return x
    
    
    def _sigmoid(self, z):
        """
        Calculates sigmoid function value.
        
        Attributes:
        -----------
        z: input value for a sigmoid function
        """
        return 1.0 / (1.0 + np.exp(-z))
    
    def _sigmoid_derivative(self, z):
        """
        Calculates sigmoid derivative function value.
        
        Attributes:
        -----------
        z: input value for a sigmoid derivative function
        """
        return self._sigmoid(z)*(1 - self._sigmoid(z))
    
    def _cost_derivative(self, output_activations, y):
        """
        Vector of partial derivatives of the loss function 
        with respect to activations in the output layer.
        
        Attributes:
        -----------
        output_activations: activations in the output layer
        y: desired output of a neural network
        """
        return output_activations - y
    
    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
    
# Notes:
# - when z is a vector (in sigmoid function) function will return vector where sigmoid function is applied element-wise

In [3]:
neuralNetwork = NeuralNetwork([100, 50, 20, 10])
# neuralNetwork.learn() - this is how to init learning in a neural network

We won't start the network to learn digits because data preparation would take some time to program. The intention was to get a deep understanding of how the neural network learns from training examples.
                                                                                                                                                                                                                                      
                                                                                                                                                                                                                                      12:00 AM September 9, 2020